In [1]:
# -*- encoding: utf-8 -*-

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
from gensim import corpora,models,similarities,utils

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
# 获取训练数据
def getTrainSet(inFile):
    # 文章标题集
    title_set = []
    # 训练集
    train_set=[]
    # 读入训练数据
    f=open(inFile)
    lines=f.readlines()
    for line in lines:
        article = line.replace('\n','').split('\t')
        title = article[0]
        title_set.append(title)
        content = article[1:]
        train_set.append(content)
    f.close()
    return (title_set,train_set)

In [6]:
def simTFIDF(test_data,mdlFile,dicFile,idxFile):
    #载入字典
    dictionary = corpora.Dictionary.load(dicFile)
    #载入TFIDF模型和索引
    tfidfModel = models.TfidfModel.load(mdlFile)
    indexTfidf = similarities.MatrixSimilarity.load(idxFile)
    
    # 处理测试数据
    query_bow = dictionary.doc2bow(test_data)

    #使用TFIDF模型向量化
    tfidfvect = tfidfModel[query_bow]
    #TFIDF相似性
    simstfidf = indexTfidf[tfidfvect]
    
    return simstfidf

In [7]:
def simLDA(test_data,tfidfMDL,ldaMDL,dicFile,idxFile):
    #载入字典
    dictionary = corpora.Dictionary.load(dicFile)
    #载入TFIDF模型
    tfidfModel = models.TfidfModel.load(tfidfMDL)
    #载入LDA模型和索引
    ldaModel = models.LdaModel.load(ldaMDL)
    indexLDA = similarities.MatrixSimilarity.load(idxFile)
    
    # 处理测试数据
    query_bow = dictionary.doc2bow(test_data)
    #使用TFIDF模型向量化
    tfidfvect = tfidfModel[query_bow]
    #然后LDA向量化,因为我们训练时的LDA是在TFIDF基础上做的,所以用itidfvect再向量化一次
    ldavec = ldaModel[tfidfvect]

    #LDA相似性
    simlda = indexLDA[ldavec]

    return simlda

In [8]:
def getTitleIdx(simValue,TitleIdx):
    
    # 标题索引
    title_set = []
    result = []
    # 读入训练数据
    f=open(TitleIdx)
    lines=f.readlines()
    for line in lines:
        title_set.append(line.replace('\n',''))
    f.close()
    
    for item in simValue:
        idx,value = item
        print "similarity article :%s,simvalue= %s" % (title_set[idx],value)

In [9]:
def getSim():
    inFile = "./data/test.txt"
    title_set,test_set = getTrainSet(inFile)

    TFIDF_mdl = "./data/all_test_TFIDF.mdl"
    TFIDF_dic = "./data/all_test_TFIDF.dic"
    TFIDF_idx = "./data/all_test_TFIDF.idx"
    
    LDA_mdl = "./data/all_test_LDA50TOPIC.mdl"
    LDA_dic = "./data/all_test_LDA50TOPIC.dic"
    LDA_idx = "./data/all_test_LDA50TOPIC.idx"
    
    TitleIdx = "./data/TitleSet.txt"
    
    
    for i,text in enumerate(test_set):
        # 返回相似度最高的前5篇文章
        simValueLDA = sorted(enumerate(simLDA(text,TFIDF_mdl,LDA_mdl,LDA_dic,LDA_idx)),key=lambda item: -item[1])[:10]
        print "Test Article = ",title_set[i]
        print "LDA=%s" % (simValueLDA)
        getTitleIdx(simValueLDA,TitleIdx)

In [10]:
def main():
    
    getSim()
    

In [11]:
if __name__ == '__main__':
    main()